In [1]:
# These slides require the RISE plugin for Jupyter -- https://github.com/damianavila/RISE

<h1 style="padding-bottom: 1em">Asyncio for the Working Python Developer</h1>
<img src="images/Artboard_1.png" style="float: right; margin-top: 0em" />
<h3 style="padding-bottom: 4em">London Python<br/>March 2017</h3>

## Yeray Diaz
## @yera_ee

# What is asyncio?

- A core Python library for concurrent programming introduced in Python 3.4
- New **`async`**/**`await`** syntax introduced in [3.5](https://docs.python.org/3/whatsnew/3.5.html#whatsnew-pep-492) and extended in [3.6](https://docs.python.org/3/whatsnew/3.6.html#whatsnew36-pep525)
- Inspired by:
<div style="margin: auto;">[<img style="display: inline" src="images/twisted.png" width="150" />](https://www.twistedmatrix.com) [<img style="display: inline" src="images/tornado.png" width="250" />](https://www.tornadoweb.org)</div>

## What's NOT asyncio?

- Available in Python 2
- Job queue
- Magic

## When should I use asyncio?

- Loads of I/O
- High availability services
- Full duplex connections

## When should I NOT use asyncio?

- CPU bound tasks
- Depend on synchronous libraries (like ORMs)
- Stuck in Python 2

# Concurrency

- Structuring your program into independently runnable elements
- Enables **parallelism**
    - The executions of these elements at the same time
- [*Concurrency is not paralellism, it's better*](https://vimeo.com/49718712) by Rob Pike

*Asyncio* allows us to define concurrent tasks that can be scheduled to run in parallel.

On a single thread.

<img src="images/chris_pratt.gif" width="500" />

# Structuring for concurrency

- **Callbacks**: "Execute this task, and when you're done execute this other function"

- **Futures (or Promises)**: "Execute this task and pretend this object is the result"
    - Readability
    - Composability
    - Better exception handling
    - Task management

- **Coroutines**: "Execute this task, when you're done I'll be right here"
    - At explicit points call other coroutines and suspend execution, yielding to other scheduled tasks
    - Even more readable, potentially very similar to synchronous code

# Functions vs. Coroutines

In [3]:
def my_function():
    print('Executing my function')
    return 42

async def my_coroutine():
    print('Executing my coroutine')
    return 42

In [4]:
function_result = my_function()

Executing my function


In [5]:
print(function_result)

42


In [6]:
coroutine_result = my_coroutine()
# no output

In [7]:
print(coroutine_result)

<coroutine object my_coroutine at 0x10cbd0258>


A coroutine object needs to be scheduled for execution, either by:
- **`await my_coroutine()`**
- or wrapped in a **Task** using **`asyncio.ensure_future`**

Let's await it then:

In [8]:
# await my_coroutine()  --> SyntaxError

**`await`** can only be used in another coroutine. So the first time:

In [9]:
import asyncio
task = asyncio.ensure_future(my_coroutine())
task

<Task pending coro=<my_coroutine() running at <ipython-input-3-9cd02186eacb>:5>>

It's ready to be scheduled, we need to tell the **event loop** to execute it.

# Event loop

- Manages and distributes the execution of different tasks.
- Uses OS signals to detect availability on socket connections.

In [10]:
loop = asyncio.get_event_loop()
loop.run_until_complete(task)

Executing my coroutine


42

It's now **done** and we can retrieve it's result at any time.

In [11]:
task.done(), task.result()

(True, 42)

## Summary of concepts

- **Coroutines**, are asynchronous functions
- **Tasks**, wrap coroutines to be executed
- **Event loop**, schedule the execution of tasks

## Scheduling tasks

Tasks can be grouped and scheduled at the same time.

- "[**gather**](https://docs.python.org/3/library/asyncio-task.html#asyncio.gather) returns a future aggregating results from the given coroutine objects or futures"

In [12]:
async def foo():
    print('Executing foo')
    return 1

async def bar():
    print('Executing bar')
    return 2

In [13]:
# group the tasks into a single task
grouped_task = asyncio.gather(foo(), bar())
# run the grouped task until complete
results = loop.run_until_complete(grouped_task)
print("The results are: {}".format(results))

Executing foo
Executing bar
The results are: [1, 2]


It does not guarantee order of execution, **bar** may execute before **foo**, depending on the event loop's whim

In [14]:
async def foo():
    print('Executing foo')
    await asyncio.sleep(0)
    print('foo back from sleeping')
    return 1

async def bar():
    print('Executing bar')
    await asyncio.sleep(0)
    print('bar back from sleeping')
    return 2

**asyncio.sleep** is a coroutine that waits for the given number of seconds.

We do not call **bar** from **foo** or viceversa.

We're not using **ensure_future** in the coroutine, a coroutine can **await** another coroutine without wrapping it in a **Task**.

In [15]:
grouped_task = asyncio.gather(foo(), bar())
results = loop.run_until_complete(grouped_task)
print("The results are: {}".format(results))

Executing foo
Executing bar
foo back from sleeping
bar back from sleeping
The results are: [1, 2]


<img src="images/foo_bar_await_2.png" width="1000" />

In [16]:
async def foo():
    print('Executing foo')
    # explicit context switch to a scheduled task
    await asyncio.sleep(0)
    print('foo back from sleeping')
    # task is done, implicit context switch to bar
    return 1

async def bar():
    print('Executing bar')
    # explicit context switch to a scheduled task
    await asyncio.sleep(0)
    print('bar back from sleeping')
    return 2
    # all tasks are done, the gather task is done

- **`await`** will suspend the coroutine and yield to the event loop which will give control to other scheduled tasks.

-  the coroutine will resume at the **`await`** point when the event loop yields control back to it.

- Upon returning or raising and exception the task is *done* also yielding control to the event loop.

# Sleepy?

<img src="images/coffee_break.gif" width="700" />

<h2 style="padding-bottom: 2em">Let's fetch some URLs</h2>
## Using [requests](http://docs.python-requests.org/en/master/)

In [17]:
import time
from datetime import datetime
from contextlib import contextmanager
from urllib.parse import urlparse
import requests

@contextmanager
def benchmark(description=''):
    start = time.time()
    yield
    print('{} took: {:.3f} seconds'.format(
        description, time.time() - start))

def get_url(response):
    try:
        url = response.url_obj
    except AttributeError:
        url = response.url
    
    return urlparse(str(url)).netloc

def get_status(response):
    try:
        return response.status
    except AttributeError:
        return response.status_code
    
    
show_timestamps = False
def print_response(response, elapsed):
    now = datetime.now()
    msg = '{:25.25} {} {} {:.3f} seconds'.format(
            get_url(response), get_status(response),
            response.reason, elapsed)
    if show_timestamps:
         msg = '[{:%H:%M:%S}.{:03.0f}] {}'.format(
            now, now.microsecond / 1000, msg)
    print(msg)
    
def print_responses(responses):
    for response, elapsed in responses:
        print_response(response, elapsed)

In [18]:
def fetch(url):
    start = time.time()
    response = requests.get(url)
    return response, time.time() - start

In [19]:
def main(urls):
    start = time.time()
    for url in urls:
        response, elapsed = fetch(url)
        print_response(response, elapsed)
    print('\nFetching all results: {:.3f} seconds'.format(
        time.time() - start))

In [20]:
urls = ['https://www.python.org',
        'http://python-requests.org']
main(urls)

www.python.org            200 OK 0.144 seconds
docs.python-requests.org  200 OK 0.427 seconds

Fetching all results: 0.572 seconds


## Using asyncio + [aiohttp](http://aiohttp.readthedocs.io/en/stable/):

In [21]:
import aiohttp
import logging

logging.basicConfig(format='%(message)s')
log = logging.getLogger(__name__)

In [22]:
async def fetch_coro(session, url):
    start = time.time()
    response = await session.get(url)
    response.close()  # required by aiohttp
    return response, time.time() - start

In [23]:
async def main_coro(urls):
    session = aiohttp.ClientSession(loop=loop)
    start = time.time()
    for url in urls:
        response, elapsed = await fetch_coro(session, url)
        print_response(response, elapsed)
        
    print('\nFetching all results took: {:.3f} seconds'.format(
        time.time() - start))

    session.close()

In [24]:
urls = ['https://python.org', 'https://aiohttp.readthedocs.io/']
loop.run_until_complete(main_coro(urls))

www.python.org            200 OK 0.418 seconds
aiohttp.readthedocs.io    200 OK 0.576 seconds

Fetching all results took: 0.994 seconds


Same result as **requests**.

In [25]:
async def main_coro(urls):
    session = aiohttp.ClientSession(loop=loop)
    start = time.time()
    for url in urls:
        response, elapsed = await fetch_coro(session, url)
        print_response(response, elapsed)
        
    print('\nFetching all results took: {:.3f} seconds'.format(
        time.time() - start))

    session.close()

On **`await`** there are no other scheduled tasks, so the event loop just waits until the request is finished and moves forward.

In [26]:
async def main_coro_gather(urls):
    session = aiohttp.ClientSession(loop=loop)
    coros = [fetch_coro(session, url) for url in urls]

    start = time.time()
    responses = await asyncio.gather(*coros)
    print_responses(responses)
    print('\nFetching all results took: {:.3f} seconds'.format(
        time.time() - start))
    session.close()

In [27]:
urls = ['https://python.org', 'https://aiohttp.readthedocs.io/']
loop.run_until_complete(main_coro_gather(urls))

www.python.org            200 OK 0.395 seconds
aiohttp.readthedocs.io    200 OK 0.443 seconds

Fetching all results took: 0.456 seconds


That's a few ms over the slowest request!

## a few more URLs...

In [28]:
urls = [
    'https://python.org', 'https://aiohttp.readthedocs.io/',
    'http://python-requests.org','https://github.com',
    'https://news.ycombinator.com/',
    'https://www.meetup.com/LondonPython/',
]
main(urls)

www.python.org            200 OK 0.428 seconds
aiohttp.readthedocs.io    200 OK 0.504 seconds
docs.python-requests.org  200 OK 0.291 seconds
github.com                200 OK 0.521 seconds
news.ycombinator.com      200 OK 0.216 seconds
www.meetup.com            200 OK 0.889 seconds

Fetching all results: 2.853 seconds


In [29]:
loop.run_until_complete(main_coro_gather(urls))

www.python.org            200 OK 0.370 seconds
aiohttp.readthedocs.io    200 OK 0.492 seconds
docs.python-requests.org  200 OK 0.296 seconds
github.com                200 OK 0.382 seconds
news.ycombinator.com      200 OK 0.500 seconds
www.meetup.com            200 OK 0.246 seconds

Fetching all results took: 0.514 seconds


<img src="images/roadrunner.gif" width="300" />

## One subtle difference

In [30]:
urls = [
    'https://python.org', 'https://aiohttp.readthedocs.io/',
    'http://python-requests.org','https://github.com',
    'https://news.ycombinator.com/',
    'https://www.meetup.com/LondonPython/',
]
show_timestamps = True
main(urls)

[13:47:17.653] www.python.org            200 OK 0.425 seconds
[13:47:18.100] aiohttp.readthedocs.io    200 OK 0.446 seconds
[13:47:18.413] docs.python-requests.org  200 OK 0.312 seconds
[13:47:18.901] github.com                200 OK 0.488 seconds
[13:47:19.056] news.ycombinator.com      200 OK 0.155 seconds
[13:47:19.540] www.meetup.com            200 OK 0.483 seconds

Fetching all results: 2.312 seconds


In [31]:
loop.run_until_complete(main_coro_gather(urls))

[13:47:25.445] www.python.org            200 OK 0.371 seconds
[13:47:25.445] aiohttp.readthedocs.io    200 OK 0.427 seconds
[13:47:25.445] docs.python-requests.org  200 OK 0.289 seconds
[13:47:25.446] github.com                200 OK 0.457 seconds
[13:47:25.446] news.ycombinator.com      200 OK 0.140 seconds
[13:47:25.446] www.meetup.com            200 OK 0.321 seconds

Fetching all results took: 0.459 seconds


- The synchronous version prints the response immediately after the request returns.
- The asynchronous version prints all of them once they're **all** finished.
- What if one takes a very long time?

## Scheduling tasks with `as_completed`

- [**`as_completed`**](https://docs.python.org/3.6/library/asyncio-task.html#asyncio.as_completed) returns an iterator whose values, when waited for, are **`Future`** instances.

In [32]:
async def main_coro_as_completed(urls):
    session = aiohttp.ClientSession(loop=loop)
    coros = [fetch_coro(session, url) for url in urls]
    start = time.time()
    for value in asyncio.as_completed(coros):
        response, elapsed = await value  # always yields completed Futures
        print_response(response, elapsed)
    print('\nFetching all results took: {:.3f} seconds'.format(
        time.time() - start))
    session.close()

In [33]:
loop.run_until_complete(main_coro_as_completed(urls))

[13:47:31.884] news.ycombinator.com      200 OK 0.142 seconds
[13:47:32.014] docs.python-requests.org  200 OK 0.272 seconds
[13:47:32.092] www.meetup.com            200 OK 0.350 seconds
[13:47:32.125] github.com                200 OK 0.384 seconds
[13:47:32.130] www.python.org            200 OK 0.389 seconds
[13:47:32.187] aiohttp.readthedocs.io    200 OK 0.456 seconds

Fetching all results took: 0.458 seconds


- The results are printed as they arrive.
- The values yielded by the iterator correspond to completed Futures.
- You must **`await`** them but they return immediately.

In [34]:
show_timestamps = False

## Oops...

In [35]:
urls = [
    'https://python.org',
    'https://aiohttp.readthedocs.io/',
    'http://python-requests.org',
    'http://not.a.thing/',  # <<
    'https://github.com',
    'https://news.ycombinator.com/',
    'https://www.meetup.com/LondonPython/',
]

### **`requests`**

In [36]:
try:
    main(urls)
except Exception as e:
    print(e)

www.python.org            200 OK 0.411 seconds
aiohttp.readthedocs.io    200 OK 0.581 seconds
docs.python-requests.org  200 OK 0.291 seconds
HTTPConnectionPool(host='not.a.thing', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x10daa47b8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))


### **`gather`**

In [37]:
try:
    loop.run_until_complete(main_coro_gather(urls))
except Exception as e:
    print(e)

[Errno 8] Cannot connect to host not.a.thing:80 ssl:False [nodename nor servname provided, or not known]


### **`as_completed`**

In [38]:
try:
    loop.run_until_complete(main_coro_as_completed(urls))
except Exception as e:
    print(e)

[Errno 8] Cannot connect to host not.a.thing:80 ssl:False [nodename nor servname provided, or not known]


# Error handling

Using **`requests`**:

In [39]:
def main(urls):
    start = time.time()
    responses = []
    for url in urls:
        try:
            response, elapsed = fetch(url)
            print_response(response, elapsed)
        except Exception as e:
            print('Error retrieving URL {}: {}'.format(
                url, e.__class__.__name__))

    print('Fetching all results: {:.3f} seconds\n'.format(
        time.time() - start))
    print_responses(responses)

In [40]:
main(urls)

www.python.org            200 OK 0.424 seconds
aiohttp.readthedocs.io    200 OK 0.556 seconds
docs.python-requests.org  200 OK 0.379 seconds
Error retrieving URL http://not.a.thing/: ConnectionError
github.com                200 OK 0.453 seconds
news.ycombinator.com      200 OK 0.155 seconds
www.meetup.com            200 OK 0.469 seconds
Fetching all results: 2.442 seconds



### In asyncio:

- Exceptions bubble in the exact same way

In [41]:
async def main_coro_as_completed(urls):
    session = aiohttp.ClientSession(loop=loop)
    coros = [fetch_coro(session, url) for url in urls]
    start = time.time()
    for value in asyncio.as_completed(coros):
        try:
            response, elapsed = await value
            print_response(response, elapsed)
        except Exception as e:
            print('Error retrieving URL: {}'.format(
                e.__class__.__name__))

    print('\nFetching all results took: {:.3f} seconds'.format(
        time.time() - start))
    session.close()

In [42]:
loop.run_until_complete(main_coro_as_completed(urls))

Error retrieving URL: ClientOSError
www.meetup.com            200 OK 0.251 seconds
docs.python-requests.org  200 OK 0.290 seconds
www.python.org            200 OK 0.398 seconds
github.com                200 OK 0.401 seconds
aiohttp.readthedocs.io    200 OK 0.457 seconds
news.ycombinator.com      200 OK 1.292 seconds

Fetching all results took: 1.294 seconds


### Using **`gather`**

In [43]:
async def main_coro_gather(urls):
    session = aiohttp.ClientSession(loop=loop)
    coros = [fetch_coro(session, url) for url in urls]
    start = time.time()
    try:
        responses = await asyncio.gather(*coros)
    except Exception as e:
        print('Error retrieving URLs: {}'.format(e.__class__.__name__))
    else:
        print('Fetching all results took: {:.3f} seconds\n'.format(
            time.time() - start))

        print_responses(responses)
    finally:
        session.close()

In [45]:
loop.run_until_complete(main_coro_gather(urls))

Error retrieving URLs: ClientOSError


What about requests that **didn't** fail?

- **`gather`** creates grouped task from many others that when awaited returns their **results**.
- If these results are exceptions they are raised by default.
- To prevent this, it accepts an argument **`return_exceptions`**:
    - "If `true`, exceptions in the tasks are treated the same as successful results, and gathered in the result list"

In [46]:
async def main_coro_gather(urls):
    session = aiohttp.ClientSession(loop=loop)
    coros = [fetch_coro(session, url) for url in urls]
    start = time.time()
    results = await asyncio.gather(
        *coros, return_exceptions=True)  # does not raise
    for url, result in zip(urls, results):
        if isinstance(result, Exception):
            print('Error retrieving URL {}: {}'.format(
                url, result.__class__.__name__))
        else:
            print_response(*result)

    session.close()

In [47]:
loop.run_until_complete(main_coro_gather(urls))

www.python.org            200 OK 0.381 seconds
aiohttp.readthedocs.io    200 OK 0.446 seconds
docs.python-requests.org  200 OK 0.348 seconds
Error retrieving URL http://not.a.thing/: ClientOSError
github.com                200 OK 0.393 seconds
news.ycombinator.com      200 OK 1.281 seconds
www.meetup.com            200 OK 0.335 seconds


# "I could do that with threads/processes"

<img src="images/thatstrue.gif" width="700" />

In [48]:
import concurrent.futures
start = time.time()

In [49]:
def main_threads():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = [executor.submit(fetch, url) for url in urls]

    print('Fetching all results took: {:.3f} seconds\n'.format(
            time.time() - start))
    successes = []
    for url, result in zip(urls, results):
        try:
            successes.append(result.result())
        except Exception as e:
            print('Error retrieving URL {}: {}'.format(
                url, e.__class__.__name__))

    print_responses(successes)

In [51]:
main_threads()

Fetching all results took: 4.850 seconds

Error retrieving URL http://not.a.thing/: ConnectionError
www.python.org            200 OK 0.405 seconds
aiohttp.readthedocs.io    200 OK 0.484 seconds
docs.python-requests.org  200 OK 0.309 seconds
github.com                200 OK 0.516 seconds
news.ycombinator.com      200 OK 0.162 seconds
www.meetup.com            200 OK 0.506 seconds


## But...
- There's an overhead of creating the threads
- **Coordination**

# CPU bound tasks

- Futures, `as_completed` and other concepts are borrowed from `concurrent.futures`
- You can schedule `ThreadPoolExecutor` or `ProcessPoolExecutor` as `asyncio` tasks for the event loop to handle

In [52]:
def fib(a=1, b=1):
    while True:
        yield a
        a, b = b, a + b
        
def nth_fibonacci(n):
    g = fib()
    for _ in range(n):
        num = next(g)
    return num

In [53]:
with benchmark('N-th Fibonacci number'):
    nth_fibonacci(500000)

N-th Fibonacci number took: 3.546 seconds


In [54]:
fib_task = loop.run_in_executor(
    None,  # or ThreadPoolExecutor
    nth_fibonacci, 500000)

grouped_task = asyncio.gather(fib_task, main_coro_gather(urls))
with benchmark('CPU + I/O task'):
    loop.run_until_complete(grouped_task)

www.python.org            200 OK 0.985 seconds
aiohttp.readthedocs.io    200 OK 0.817 seconds
docs.python-requests.org  200 OK 0.776 seconds
Error retrieving URL http://not.a.thing/: ClientOSError
github.com                200 OK 0.713 seconds
news.ycombinator.com      200 OK 0.319 seconds
www.meetup.com            200 OK 0.601 seconds
CPU + I/O task took: 3.548 seconds


# Let's do some crawling

<img src="images/HackerNews.png" width="800" />

- [Hacker News API](https://github.com/HackerNews/API): "Want to know the total number of comments on an article? Traverse the tree and count."

In [55]:
import json
import async_timeout

loop = asyncio.get_event_loop()
URL_TEMPLATE = "https://hacker-news.firebaseio.com/v0/item/{}.json"
FETCH_TIMEOUT = 10

class URLFetcher():
    """Provides counting of URL fetches for a particular task.

    """

    def __init__(self):
        self.fetch_counter = 0

    async def fetch(self, url, session=None):
        """Fetch a URL using aiohttp returning parsed JSON response.

        As suggested by the aiohttp docs we reuse the session.

        """
        session = aiohttp.ClientSession(loop=loop) if session is None else session
        async with session:
            with async_timeout.timeout(FETCH_TIMEOUT):
                self.fetch_counter += 1
                async with session.get(url) as response:
                    return await response.json()

In [56]:
fetcher = URLFetcher()
url = URL_TEMPLATE.format(13955074)
result = loop.run_until_complete(fetcher.fetch(url))
print(json.dumps(result, indent=2))

{
  "by": "jrsinclair",
  "descendants": 1,
  "id": 13955074,
  "kids": [
    13958537
  ],
  "score": 8,
  "time": 1490441022,
  "title": "JavaScript. But less iffy",
  "type": "story",
  "url": "http://jrsinclair.com/articles/2017/javascript-but-less-iffy/"
}


### How about some recursion?

- Each item has **`kid`** items, count them as the initial number of comments
- Do the same for each of the **`kid`** items until we arrive at the leaf items with no kids and return 0
- Add the initial comment count to that of the children's and return
- Arrive at a grand total

In [57]:
async def post_number_of_comments(post_id, fetcher=None):
    url = URL_TEMPLATE.format(post_id)
    fetcher = URLFetcher() if fetcher is None else fetcher
    response = await fetcher.fetch(url)

    if response is None or 'kids' not in response:
        return 0

    number_of_comments = len(response['kids'])

    tasks = [post_number_of_comments(
        kid_id, fetcher) for kid_id in response['kids']]

    results = await asyncio.gather(*tasks)

    number_of_comments += sum(results)
    return number_of_comments

In [58]:
with benchmark('Gathering comments'):
    fetcher = URLFetcher()
    comments = loop.run_until_complete(
        post_number_of_comments(13952513, fetcher))
    print('{} comments, fetches: {}'.format(
        comments, fetcher.fetch_counter))

20 comments, fetches: 21
Gathering comments took: 2.031 seconds


In [59]:
with benchmark('Gathering comments'):
    fetcher = URLFetcher()
    comments = loop.run_until_complete(
        post_number_of_comments(13950002, fetcher))
    print('{} comments, fetches: {}'.format(
        comments, fetcher.fetch_counter))

500 comments, fetches: 501
Gathering comments took: 12.424 seconds


## The code one more time

In [60]:
async def post_number_of_comments(post_id, fetcher=None):
    url = URL_TEMPLATE.format(post_id)
    fetcher = URLFetcher() if fetcher is None else fetcher
    response = await fetcher.fetch(url)

    if response is None or 'kids' not in response:
        return 0

    number_of_comments = len(response['kids'])

    tasks = [post_number_of_comments(
        kid_id, fetcher) for kid_id in response['kids']]

    results = await asyncio.gather(*tasks)

    number_of_comments += sum(results)
    return number_of_comments

- A synchronous version would be almost identical
- To handle errors simply use **`try..except`**
- No need for synchronisation, everything is on a single thread

# Conclusions

## The good

- **`asyncio`** and the new **`async`**/**`await`** syntax have moved Python forward
- Core library: full integration with the language
- Unified API for other frameworks
- Basis for a [plethora of async libraries](https://github.com/timofurrer/awesome-asyncio)

## The bad

- All or nothing, all libraries need to be async
- Requires a permanent shift of mindset across all codebase
- Very basic API, sometimes confusing

<h1 style="text-align: center; margin-bottom: 3em;">Try it out!</h1>

<h2 style="text-align: center; margin-bottom: 3em;">Thank you! &mdash; London Python &mdash; March 2017</h2>

<h3 style="text-align: center">Yeray Diaz</h3>
<h3 style="text-align: center">@yera_ee</h3>